In [ ]:
import json
import pathlib
import random

import numpy as np
import matplotlib.pyplot as plt

from IPython import display

In [ ]:
import tensorflow as tf

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagated into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import pymedphys
from pymedphys._experimental.autosegmentation import indexing, filtering, pipeline, mask

In [ ]:
mask_expansion = 5

In [ ]:
# Create masks for the following structures, in the following order
structures_to_learn = [
    'eye_left', 'eye_right', 'patient']

# Use the following to filter the slices used for training, validation,
# and testing
filters = {
    "study_set_must_have_all_of": structures_to_learn,
    "slice_at_least_one_of": [
        'lens_left', 'lens_right', 'eye_left', 'eye_right'
    ],
    "slice_must_have": ['patient'],
    "slice_cannot_have": []
}

In [ ]:
structure_uids, ct_uids = pipeline.get_filtered_uids(filters)

In [ ]:
dataset = pipeline.create_dataset(ct_uids, structures_to_learn, expansion=mask_expansion)

In [ ]:
def diagnostic_plotting(x_grid, y_grid, input_array, output_array):
    plt.figure(figsize=(15,10))
    
    x_grid = x_grid.numpy()
    y_grid = y_grid.numpy()
    input_array = input_array.numpy()[:,:,0]
    output_array = output_array.numpy()
    
    for i, structure in enumerate(structures_to_learn[0:-1]):
        if structure.endswith('left'):
            colour = 'r'
        elif structure.endswith('right'):
            colour = 'b'
        else:
            raise ValueError("Expected either left or right")
            
        if structure.startswith('lens'):
            colour += '--'
        elif structure.startswith('eye'):
            colour += '-'
        else:
            raise ValueError("Expected either eye or lens")

        contours = mask.get_contours_from_mask(
            x_grid, y_grid, output_array[:,:,i])
        for contour in contours:
            plt.plot(*contour.T, colour)
            
    
    contours = mask.get_contours_from_mask(
        x_grid, y_grid, output_array[:,:,-1])
    for contour in contours:
        plt.plot(*contour.T, 'k--')
        
    plt.axis('equal')
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    windowed = np.copy(input_array)

    vmin = 900
    vmax = 1200
    windowed[windowed<vmin] = vmin
    windowed[windowed>vmax] = vmax

    plt.pcolormesh(x_grid, y_grid, windowed, shading="nearest")
    plt.colorbar()
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    
    return xlim, ylim

In [ ]:
for ct_uid, x_grid, y_grid, input_array, output_array in dataset:
    ct_uid = ct_uid.numpy().decode()
    
    display.display(display.Markdown(f"## {ct_uid}"))
    xlim, ylim = diagnostic_plotting(x_grid, y_grid, input_array, output_array)
    plt.show()
    
    print(np.mean(xlim))